In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
aux=postulantes[postulantes['edad']<=90].edad.mean()
postulantes.edad=postulantes.edad.apply(lambda x: x if x<=90 else aux)
postulantes.edad.value_counts()

30.721083    26727
26.000000    26367
25.000000    25860
27.000000    25822
24.000000    25409
23.000000    24885
22.000000    23254
28.000000    23022
29.000000    21917
21.000000    21265
30.000000    20007
31.000000    18987
20.000000    17853
32.000000    17150
33.000000    15119
34.000000    14118
19.000000    13569
35.000000    13129
36.000000    12463
37.000000    11675
38.000000    10786
39.000000    10065
40.000000     8987
41.000000     8045
42.000000     7292
18.000000     6961
43.000000     6595
44.000000     5593
45.000000     4894
46.000000     4548
             ...  
17.000000      302
64.000000      227
65.000000      171
66.000000      143
67.000000      124
68.000000       83
69.000000       61
70.000000       34
71.000000       21
72.000000       20
73.000000       15
75.000000       13
74.000000       12
16.000000        9
76.000000        6
78.000000        6
77.000000        3
79.000000        3
12.000000        2
13.000000        1
80.000000        1
83.000000   

In [5]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [6]:
vistas_por_area=vistas.copy()
vistas_por_area['vistas_por_area']=0;
vistas_por_area=pd.merge(vistas_por_area,avisos,on='idaviso', how='left')
vistas_por_area=vistas_por_area.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_por_area.head()

,idpostulante,nombre_area,vistas_por_area
0,0005E,Tecnologia / Sistemas,2
1,00Lkv,Comercio Exterior,2
2,00dMd,Comercio Exterior,4
3,01QAq,Contabilidad,1
4,021OM,Comercio Exterior,5


In [7]:
vistas_por_nivel=vistas.copy()
vistas_por_nivel['vistas_por_nivel']=0;
vistas_por_nivel=pd.merge(vistas_por_nivel,avisos,on='idaviso', how='left')
vistas_por_nivel=vistas_por_nivel.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})
vistas_por_nivel.head()

,idpostulante,nivel_laboral,vistas_por_nivel
0,0005E,Otro,2
1,00Lkv,Otro,2
2,00dMd,Otro,4
3,01QAq,Otro,1
4,021OM,Otro,5


In [8]:
vistas_por_tipo=vistas.copy()
vistas_por_tipo['vistas_por_tipo']=0;
vistas_por_tipo=pd.merge(vistas_por_tipo,avisos,on='idaviso', how='left')
vistas_por_tipo=vistas_por_tipo.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_por_tipo.head()

,idpostulante,tipo_de_trabajo,vistas_por_tipo
0,0005E,Full-time,2
1,00Lkv,Full-time,2
2,00dMd,Full-time,4
3,01QAq,Full-time,1
4,021OM,Full-time,5


In [9]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [10]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [11]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [12]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [13]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [14]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [15]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//4).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2.sample(10)

8255667


,idaviso,idpostulante,sepostulo
3703159,1112464211,ak4BaA9,0
4918592,1000380240,rmP96Wd,0
4069957,1112396585,ekNQARp,0
5424731,1112221656,QNPZE8e,0
2016081,1112165365,owzeOok,0
4956924,1112432964,aXAKx0,0
6263359,1112390967,YK9GWY,0
1493287,1112308839,EzEjwmJ,0
634220,1112439158,MVPp0Qr,0
650580,1112473443,PmqWJBR,0


In [16]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [17]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

14860201
14852872


In [18]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad
9392024,1112279347,MVPrqMe,0,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,3.0,NaN,NaN
12821517,1111745880,ow5LXae,0,FEM,25.0,-,-,Graduado,-,Graduado,...,True,False,False,False,False,False,False,NaN,NaN,NaN
12101984,1111131531,6v3rxO,1,MASC,45.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,17.0,NaN,NaN
2347256,1112280429,NzewELB,0,MASC,23.0,-,-,En Curso,-,Graduado,...,False,False,False,False,False,False,False,NaN,NaN,NaN
1639162,1112250944,3N2AdqP,0,FEM,22.0,-,-,-,-,-,...,False,False,True,True,False,False,False,61.0,NaN,NaN
485275,1112307266,5562Zw,1,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,3.0,NaN,NaN
7805076,1112437647,wVODK8m,0,FEM,26.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,4.0,NaN,NaN
9960032,1112343030,4rPp92q,0,FEM,52.0,-,-,-,-,-,...,True,False,True,False,False,True,False,45.0,NaN,NaN
10302705,1111951148,EmXwD8,0,MASC,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,2.0,NaN,NaN
5016178,1112253432,jkjdN1j,1,FEM,25.0,-,-,En Curso,-,Graduado,...,False,False,False,False,False,False,False,536.0,NaN,NaN


In [19]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
4007531,1112314953,VNrbO4z,0,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,True,False,True,22.0,0.0,0.000000,False
13725161,1112228947,bOjwOq4,0,FEM,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,33.0,0.0,0.000000,False
12628876,1112216850,Y1KWYX,0,MASC,37.0,-,-,-,-,-,...,False,False,False,False,True,False,28.0,0.0,0.000000,False
6378122,1112460220,LN8kvm5,0,FEM,25.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,145.0,0.0,0.000000,False
3602903,1111218850,kPNmrPj,1,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,35.0,1.0,0.028571,True
5155730,1112310562,lDLGvxG,0,FEM,43.0,-,-,-,-,-,...,False,False,False,False,False,False,49.0,0.0,0.000000,False
11263218,1112211451,ow2q4rr,1,MASC,35.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,1.0,0.0,0.000000,False
7610403,1112369137,Pma2Daj,0,MASC,33.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,65.0,0.0,0.000000,False
5267249,1111351367,ak4ZBRZ,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.000000,False
13433137,1112205556,DrzDrY5,0,MASC,26.0,-,-,-,-,-,...,False,False,False,False,False,False,5.0,0.0,0.000000,False


In [20]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Abastecimiento,1
1,0z5Dmrd,Administración,1
2,0z5Dmrd,Almacén / Depósito / Expedición,1
3,0z5Dmrd,Atención al Cliente,1
4,0z5Dmrd,Auditoría,1


In [21]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
1,0z5Dmrd,Junior,3
2,0z5Dmrd,Otro,1
3,0z5Dmrd,Senior / Semi-Senior,16
4,0z5JW1r,Gerencia / Alta Gerencia / Dirección,2


In [22]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,20
1,0z5Dmrd,Part-time,1
2,0z5JW1r,Full-time,24
3,0z5VvGv,Full-time,54
4,0z5VvGv,Part-time,11


In [23]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones=pd.merge(postulaciones,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
364323,1112306493,ZxVjWV,1,FEM,29.000000,-,-,-,-,-,...,354.0,0.0,0.000000,False,20,189,157,69.0,184.0,184.0
13138431,1111682722,A3RoAZ1,0,FEM,24.000000,-,-,-,-,Graduado,...,411.0,0.0,0.000000,False,3,22,19,33.0,72.0,72.0
9216497,1112254274,6V6RZO,0,MASC,31.000000,-,-,-,-,-,...,52.0,0.0,0.000000,False,1,147,60,0.0,33.0,33.0
5313121,1112430811,KBrEb9p,0,NO_DECLARA,30.721083,-,-,-,-,-,...,27.0,0.0,0.000000,False,0,12,11,0.0,16.0,16.0
6592133,1112290385,ow5ELkL,0,FEM,26.000000,-,-,-,-,Graduado,...,171.0,0.0,0.000000,False,0,146,132,0.0,136.0,136.0
7102307,1112442557,xkP53Ka,0,MASC,34.000000,-,-,-,-,-,...,1.0,0.0,0.000000,False,0,20,16,0.0,1.0,1.0
8433605,1112392684,NqAAJl,1,FEM,29.000000,-,-,-,-,Graduado,...,7.0,0.0,0.000000,False,7,39,30,0.0,7.0,7.0
3870915,1112202359,vVDlKKo,1,MASC,25.000000,-,-,-,-,-,...,19.0,0.0,0.000000,False,37,161,123,4.0,16.0,16.0
8393442,1112280944,qekOkb1,1,FEM,29.000000,-,-,-,-,Graduado,...,30.0,0.0,0.000000,False,14,55,9,12.0,29.0,29.0
4889864,1112415823,xkv9G3K,0,FEM,29.000000,-,-,Graduado,-,Graduado,...,0.0,0.0,0.000000,False,1,13,6,0.0,0.0,0.0


In [24]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  13715196 Test:  721853


In [29]:
c=0
gc.collect()

0

In [30]:
c=lgb.LGBMClassifier(learning_rate=0.05,objective='binary',num_leaves=12000,max_depth=18,n_estimators=1200,colsample_bytree=0.9,n_jobs=-1,random_state=0,silent=False)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.658937
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.627885
[3]	valid_0's binary_logloss: 0.598581
[4]	valid_0's binary_logloss: 0.570739
[5]	valid_0's binary_logloss: 0.545302
[6]	valid_0's binary_logloss: 0.521878
[7]	valid_0's binary_logloss: 0.501163
[8]	valid_0's binary_logloss: 0.481248
[9]	valid_0's binary_logloss: 0.463659
[10]	valid_0's binary_logloss: 0.447266
[11]	valid_0's binary_logloss: 0.431328
[12]	valid_0's binary_logloss: 0.416543
[13]	valid_0's binary_logloss: 0.402815
[14]	valid_0's binary_logloss: 0.390541
[15]	valid_0's binary_logloss: 0.37852
[16]	valid_0's binary_logloss: 0.368105
[17]	valid_0's binary_logloss: 0.357541
[18]	valid_0's binary_logloss: 0.34764
[19]	valid_0's binary_logloss: 0.338365
[20]	valid_0's binary_logloss: 0.329712
[21]	valid_0's binary_logloss: 0.321607
[22]	valid_0's binary_logloss: 0.313973
[23]	valid_0's binary_logloss: 0.307529
[24]	valid_0's binary_logloss:

In [31]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9855701260478776


In [32]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV32.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV32.pkl']

In [33]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['lo_vio']=(prediccion['cantidad']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion=pd.merge(prediccion,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
63365,10775,1111926238,6kLYpO,FEM,40.000000,-,-,-,-,Graduado,...,NaN,NaN,0.000000,False,0.0,18.0,14.0,0.0,0.0,0.0
97135,96844,1112495904,1qGoab,MASC,39.000000,-,-,-,-,Graduado,...,8.0,NaN,0.000000,False,1.0,22.0,14.0,0.0,8.0,8.0
58706,7572,1111720481,6RZYx3,FEM,33.000000,-,-,En Curso,-,Graduado,...,45.0,NaN,0.000000,False,1.0,15.0,0.0,0.0,24.0,24.0
94908,94269,1112483543,E0AxZb,MASC,42.000000,-,-,-,-,Graduado,...,14.0,NaN,0.000000,False,0.0,16.0,13.0,2.0,14.0,14.0
42697,1585,1110921809,ZGbB6E,FEM,43.000000,-,En Curso,-,Graduado,Graduado,...,8.0,NaN,0.000000,False,0.0,13.0,1.0,0.0,8.0,8.0
10745,88592,1112467005,mzVKdr6,MASC,26.000000,-,-,-,-,Graduado,...,NaN,NaN,0.000000,False,85.0,254.0,186.0,0.0,0.0,0.0
31990,3162,1111351367,ZDr1ZlY,FEM,20.000000,-,-,-,-,Graduado,...,732.0,1.0,0.001366,True,2.0,337.0,247.0,4.0,543.0,543.0
12314,48888,1112437188,JBe8V33,MASC,33.000000,-,Graduado,-,Graduado,Graduado,...,48.0,NaN,0.000000,False,4.0,82.0,6.0,2.0,41.0,41.0
73065,19122,1112222997,52KPzX,NO_DECLARA,30.721083,-,-,-,-,-,...,NaN,NaN,0.000000,False,1.0,14.0,8.0,0.0,0.0,0.0
11256,80709,1112465174,QNr9OEN,NO_DECLARA,30.721083,-,-,-,-,Graduado,...,139.0,1.0,0.007194,True,1.0,15.0,15.0,1.0,66.0,66.0


In [34]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[9.98971490e-01, 1.02850989e-03],
       [9.98680734e-01, 1.31926569e-03],
       [9.99510559e-01, 4.89440793e-04],
       ...,
       [9.99618201e-01, 3.81799243e-04],
       [9.99848618e-01, 1.51382091e-04],
       [9.99886584e-01, 1.13415671e-04]])

In [35]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [36]:
prediccion['sepostulo']=y_final2

In [37]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,NaN,0.0,False,1.0,16.0,0.0,4.0,24.0,24.0,0.001029
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,NaN,0.0,False,6.0,58.0,0.0,0.0,6.0,6.0,0.001319
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,NaN,0.0,False,3.0,14.0,2.0,1.0,7.0,7.0,0.000489
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,NaN,0.0,False,1.0,15.0,0.0,0.0,0.0,0.0,0.000123
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,NaN,0.0,False,2.0,16.0,3.0,0.0,3.0,3.0,0.000149


In [38]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.001029
1,1,0.001319
2,2,0.000489
3,3,0.000123
4,4,0.000149


In [39]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion64.csv",index=False)

In [40]:
prediccion.sepostulo.mean()

0.47188291998593407